In [1]:
import numpy as np
import json
import math
import copy
import pandas as pd
import dill
import os
import sys



In [5]:
BodyBrain_stats = pd.read_csv('/media/leguiart/LuisExtra/ExperimentsData2/BodyBrainQN_stats.csv')
BodyBrain_raw = pd.read_csv('/media/leguiart/LuisExtra/ExperimentsData2/BodyBrainQN.csv')

In [6]:
print(BodyBrain_stats.shape)
print(BodyBrain_raw.shape)


(2382097, 9)
(1931430, 33)


In [4]:
BodyBrain_stats.head(25)

,Indicator,Best,Worst,Average,STD,Median,Generation,Run,Method
0,fitness,0.090576,0.000000,0.027135,0.023349,0.031786,1,1,QN-MOEA
1,morphology,85.000000,2.000000,32.766667,20.673950,26.000000,1,1,QN-MOEA
2,unaligned_novelty,48.047213,21.401567,29.177554,6.587898,27.550011,1,1,QN-MOEA
3,aligned_novelty,0.082337,0.021433,0.035387,0.014831,0.033254,1,1,QN-MOEA
4,gene_diversity,5.021236,3.562090,4.099654,0.347313,4.011420,1,1,QN-MOEA
5,control_gene_div,6.791632,3.702458,4.913947,0.699827,4.826297,1,1,QN-MOEA
6,morpho_gene_div,3.765990,2.697792,3.285361,0.273798,3.232853,1,1,QN-MOEA
7,morpho_div,55.086201,27.168157,36.337462,7.219166,35.045008,1,1,QN-MOEA
8,endpoint_div,0.089809,0.033416,0.048202,0.015251,0.043067,1,1,QN-MOEA
9,trayectory_div,0.122451,0.038667,0.056652,0.021009,0.052152,1,1,QN-MOEA


In [5]:
BodyBrain_stats.isnull().sum()

Indicator     0
Best          0
Worst         0
Average       0
STD           0
Median        0
Generation    0
Run           0
Method        0
dtype: int64

In [6]:
BodyBrain_raw.head(10)

,id,generation,run,md5,fitness,unaligned_novelty,aligned_novelty,gene_diversity,control_gene_div,morpho_gene_div,...,morphology_passive,control_cppn_nodes,control_cppn_edges,control_cppn_ws,morpho_cppn_nodes,morpho_cppn_edges,morpho_cppn_ws,simplified_gene_div,simplified_gene_ne_div,simplified_gene_nws_div
0,0,1,1,2f5e6549d406f0d2b1989d4919a85db3,0.041018,31.643533,0.038006,3.821216,3.980650,3.661783,...,17,12,20,1.791111,12,20,1.203642,4.732056,3.773725,2.273843
1,1,1,1,65f920223fde124b34c67b1f1d374e04,0.031262,27.627683,0.035602,3.812028,4.707445,2.916611,...,33,12,20,3.330835,12,20,1.258988,5.041880,4.148330,2.273843
2,2,1,1,5ab3dd38585d9993b0fa1a8c7f6a2219,0.039115,24.793155,0.036063,4.346910,5.295437,3.398384,...,26,11,19,-3.877098,12,20,3.292124,6.788769,6.167620,2.327004
3,3,1,1,9318f8fd880985e9c6d4668be397d44d,0.000000,46.988429,0.022144,3.801821,4.640031,2.963611,...,15,12,20,5.553271,12,20,7.136739,8.387798,7.829881,2.273843
4,4,1,1,33836e630c181d0660fe1e6e6ce90429,0.035757,21.401567,0.035569,4.355935,5.696488,3.015383,...,30,11,19,0.535062,12,20,-0.346971,4.874584,3.980953,2.327004
5,5,1,1,9b8080d601335b905a1939f020e35b4d,0.039482,21.510564,0.038868,4.705880,6.331409,3.080350,...,26,12,20,2.736047,12,20,3.842708,5.423027,4.616755,2.273843
6,6,1,1,b8b7be23d2e6caeaf18e3c460d3afe9a,0.009159,22.221494,0.028456,4.163032,5.003588,3.322476,...,23,11,17,-0.071767,11,19,10.325751,9.817893,9.315540,3.128891
7,7,1,1,b76704ae5d7956f2e7c019fdab617089,0.090576,26.111081,0.079928,3.800042,3.868909,3.731174,...,49,11,18,-3.552462,11,18,0.843492,6.560180,5.775205,3.037440
8,8,1,1,8d4e761dd4903da314a11d2d3c07df8c,0.017559,25.800924,0.045029,4.428200,5.186003,3.670397,...,21,12,20,-0.845142,12,20,3.821635,5.576168,4.731659,2.273843
9,9,1,1,04a1cfdc0c31946dd6fec6773a074000,0.038276,22.958567,0.037253,3.921963,4.684058,3.159867,...,21,10,15,2.326769,10,16,-2.534770,7.860526,5.625912,5.670266


In [7]:
BodyBrain_raw[BodyBrain_raw['fitness'] <= 0.5]

,id,generation,run,md5,fitness,unaligned_novelty,aligned_novelty,gene_diversity,control_gene_div,morpho_gene_div,...,morphology_passive,control_cppn_nodes,control_cppn_edges,control_cppn_ws,morpho_cppn_nodes,morpho_cppn_edges,morpho_cppn_ws,simplified_gene_div,simplified_gene_ne_div,simplified_gene_nws_div
0,0,1,1,2f5e6549d406f0d2b1989d4919a85db3,4.101798e-02,31.643533,0.038006,3.821216,3.980650,3.661783,...,17,12,20,1.791111,12,20,1.203642,4.732056,3.773725,2.273843
1,1,1,1,65f920223fde124b34c67b1f1d374e04,3.126226e-02,27.627683,0.035602,3.812028,4.707445,2.916611,...,33,12,20,3.330835,12,20,1.258988,5.041880,4.148330,2.273843
2,2,1,1,5ab3dd38585d9993b0fa1a8c7f6a2219,3.911505e-02,24.793155,0.036063,4.346910,5.295437,3.398384,...,26,11,19,-3.877098,12,20,3.292124,6.788769,6.167620,2.327004
3,3,1,1,9318f8fd880985e9c6d4668be397d44d,0.000000e+00,46.988429,0.022144,3.801821,4.640031,2.963611,...,15,12,20,5.553271,12,20,7.136739,8.387798,7.829881,2.273843
4,4,1,1,33836e630c181d0660fe1e6e6ce90429,3.575664e-02,21.401567,0.035569,4.355935,5.696488,3.015383,...,30,11,19,0.535062,12,20,-0.346971,4.874584,3.980953,2.327004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1931425,95545,1825,20,748aefb2afcf3cf6378bc04e37a4378f,2.058197e-01,1.838478,0.199825,1.081766,0.902417,1.261115,...,64,18,27,8.781646,15,20,7.732475,5.973867,3.537487,5.552666
1931426,95546,1825,20,e08f7fbd6f66f07ffb58587d9d9cb269,1.124610e-01,1.650000,0.134384,2.261820,2.210630,2.313010,...,16,22,38,10.143099,12,16,5.311534,8.618845,3.752761,8.236886
1931427,95547,1825,20,1b9a4ffd9a6298f8856c7761854f0dfc,5.685939e-07,7.424621,0.136757,1.010845,0.907913,1.113776,...,57,20,29,10.781646,14,19,6.932475,4.314471,2.433556,3.885227
1931428,95548,1825,20,74d89eac8678e4c6d10961a71584c774,3.116526e-01,6.329964,0.272026,0.998057,0.878143,1.117971,...,56,19,28,8.937638,15,21,7.832475,5.395913,3.032886,4.959354


In [8]:
BodyBrain_raw.isnull().sum()

id                         0
generation                 0
run                        0
md5                        0
fitness                    0
unaligned_novelty          0
aligned_novelty            0
gene_diversity             0
control_gene_div           0
morpho_gene_div            0
morpho_div                 0
endpoint_div               0
trayectory_div             0
inipoint_x                 0
inipoint_y                 0
inipoint_z                 0
endpoint_x                 0
endpoint_y                 0
endpoint_z                 0
trayectory_x               0
trayectory_y               0
trayectory_z               0
morphology_active          0
morphology_passive         0
control_cppn_nodes         0
control_cppn_edges         0
control_cppn_ws            0
morpho_cppn_nodes          0
morpho_cppn_edges          0
morpho_cppn_ws             0
simplified_gene_div        0
simplified_gene_ne_div     0
simplified_gene_nws_div    0
dtype: int64

In [16]:
#partitioning_indexes = BodyBrain_raw.index[BodyBrain_raw['id'] == 0].tolist()
#partitioning_indexes += [len(BodyBrain_raw) - 1]
#grouped_by_run = [BodyBrain_raw.iloc[partitioning_indexes[i]:partitioning_indexes[i+1],:] for i in range(len(partitioning_indexes) - 1)]
grouped_by_run = [BodyBrain_raw[BodyBrain_raw['run'] == i] for i in range(1, 21)]

In [19]:
for i, run_df in enumerate(grouped_by_run):
    if len(run_df) > 90000:
        print(i + 1)
    #print(len(run_df))

1
2
4
6
8
10
12
14
16
18
20


In [11]:
for i, run_df in enumerate(grouped_by_run):
    if len(run_df) > 90000:
        # First remove duplicates
        df = run_df.copy()
        df = df.drop_duplicates(subset = ['id'], keep = 'last')
        # Then remove any row after generation 3000
        df = df.iloc[:90000, :]
        # And detect where the gen indexes are restarted
        maxGenSoFar = -np.inf
        gen_restart_indx = 0
        
        for j in range(len(df)):
            gen_restart_indx = j
            row = df.iloc[j, :]
            #print(row)
            if row['generation'] > maxGenSoFar:
                maxGenSoFar = row['generation']
            elif row['generation'] < maxGenSoFar:
                break
        df.iloc[gen_restart_indx:,1].loc['generation'] = np.array([gen for _ in range(30) for gen in range(maxGenSoFar + 1, 3001)])
        grouped_by_run[i] = df

In [12]:
cleaned_df_path = '/media/leguiart/LuisExtra/ExperimentsData2/BodyBrainQN_cleaned.csv'

grouped_by_run[0].to_csv(cleaned_df_path,mode = 'w', index=False)
for run_df in grouped_by_run[1:]:
    run_df.to_csv(cleaned_df_path, mode = 'a', header = False, index=False)

In [16]:
len(grouped_by_run)

20

In [57]:
stats_grouped_by_run = [BodyBrain_stats[BodyBrain_stats['Run'] == i] for i in range(1, 21)]

In [58]:
for run_df in stats_grouped_by_run:
    print(len(run_df))

127687
116698
111000
132053
111000
146150
111000
115107
111000
177156
111000
111333
111000
113294
111000
111222
111000
114293
111000
118104


In [59]:
g_list = []
for g in range(1, 3001):
    for _ in range(37):
        g_list += [g]

for i, run_df in enumerate(stats_grouped_by_run):
    if len(run_df) > 37 * 3000:
        df = run_df.copy()
        maxGenSoFar = -np.inf
        gen_restart_indx = []
        
        # We remove duplicates
        for j in range(len(df)):
            row = df.iloc[j, :]
            #print(row)
            if row['Generation'] > maxGenSoFar:
                maxGenSoFar = row['Generation']
            elif row['Generation'] < maxGenSoFar:
                gen_restart_indx += [j]
                maxGenSoFar = row['Generation']
                
        gen_restart_indx += [len(df) - 1]
        new_df = df.iloc[0:gen_restart_indx[0],:]
        
        for k in range(len(gen_restart_indx) - 1):
            asc_gen_df_slice = df.iloc[gen_restart_indx[k]:gen_restart_indx[k + 1], :]
            max_gen_reached = asc_gen_df_slice.iloc[-1,:]['Generation']
            max_df_size = 37 * max_gen_reached
            if len(asc_gen_df_slice) > max_df_size:
                rows_to_remove = len(asc_gen_df_slice) - max_df_size
                asc_gen_df_slice = asc_gen_df_slice.iloc[rows_to_remove - 1 :, :]
            new_df = new_df.append(asc_gen_df_slice, ignore_index=True)
        
        # new_df should have no duplicates now
        if len(new_df) > 37 * 3000:
            new_df = new_df.iloc[0 : 37 * 3000, :]

        new_df['Generation'] = np.array(g_list)
        #print(new_df)
        stats_grouped_by_run[i] = new_df.copy()
            

In [60]:
for run_df in stats_grouped_by_run:
    print(len(run_df))

111000
111000
111000
111000
111000
111000
111000
111000
111000
111000
111000
111000
111000
111000
111000
111000
111000
111000
111000
111000


In [61]:
cleaned_df_path = '/media/leguiart/LuisExtra/ExperimentsData2/BodyBrainQN_stats_cleaned.csv'

stats_grouped_by_run[0].to_csv(cleaned_df_path,mode = 'w', index=False)
for run_df in stats_grouped_by_run[1:]:
    run_df.to_csv(cleaned_df_path, mode = 'a', header = False, index=False)

In [3]:
import numpy as np
import json
import math
import copy
import pandas as pd
import dill
import os
import sys

BodyBrain_stats = pd.read_csv('/media/leguiart/LuisExtra/ExperimentsData2/BodyBrain_stats.csv')
BodyBrain_raw = pd.read_csv('/media/leguiart/LuisExtra/ExperimentsData2/BodyBrain.csv')
diversities = pd.read_csv('/media/leguiart/LuisExtra/ExperimentsData2/gene_div.csv')

In [7]:
so_diversities = diversities[diversities.Method == 'SO']
so_diversities

,Id,control_div,morpho_div,gene_div,Generation,Run,Method
0,28,4.839744,2.664725,5.583400,1,1,SO
1,7,5.995350,3.413436,6.998795,1,1,SO
2,19,5.434079,3.229738,6.393694,1,1,SO
3,23,4.140472,3.219485,5.306388,1,1,SO
4,4,4.988583,3.521787,6.181821,1,1,SO
...,...,...,...,...,...,...,...
1799695,89959,0.938223,0.230222,0.969852,3000,20,SO
1799696,89800,1.087298,0.159899,1.114267,3000,20,SO
1799697,89895,0.790429,0.262088,0.855560,3000,20,SO
1799698,89955,0.984558,0.168591,1.009124,3000,20,SO


In [8]:
so_divs_grouped_by_run = [so_diversities[so_diversities['Run'] == i] for i in range(1, 21)]
so_info_grouped_by_run = [BodyBrain_raw[BodyBrain_raw['run'] == i] for i in range(1, 21)]

In [9]:
for info in so_divs_grouped_by_run:
    print(len(info))

NameError: name 'so_divs_grouped_by_run' is not defined

In [15]:
parent_pop_dfs = []
for i, run_df in enumerate(so_divs_grouped_by_run):
    parent_pop_dfs += [run_df.join(so_info_grouped_by_run[i].set_index('id'), on='Id', lsuffix='_caller', rsuffix='_other')]

In [16]:
parents_df_path = '/media/leguiart/LuisExtra/ExperimentsData2/BodyBrainParents.csv'

parent_pop_dfs[0].to_csv(parents_df_path,mode = 'w', index=False)
for run_df in parent_pop_dfs[1:]:
    run_df.to_csv(parents_df_path, mode = 'a', header = False, index=False)